In [7]:
%%time
import cv2
import os
import numpy as np
from PIL import Image
import glob
from tqdm import tqdm_notebook as tqdm

step_sec = 2

for fname in tqdm(glob.glob('downloaded/*.mp4')):
    fname_w_o_extension = fname[:-4]
    video = cv2.VideoCapture(fname)
    fps = video.get(cv2.CAP_PROP_FPS)
    time_length = video.get(cv2.CAP_PROP_FRAME_COUNT)
    frame_seq = 0
    count_sec = 0
    while True:         
        frame_no = (frame_seq /(time_length*fps))
        video.set(cv2.CAP_PROP_POS_FRAMES, fps*count_sec)
        ret, frame = video.read()
        if ret:
            cv2.imwrite(f"frames/{fname_w_o_extension}_{count_sec}.jpg", frame)
            frame_seq += fps
            count_sec += step_sec
        else:
            break

<timed exec>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/40 [00:00<?, ?it/s]

CPU times: user 36min 35s, sys: 46.8 s, total: 37min 22s
Wall time: 6min 9s


In [ ]:
# save embeddings
from clip_embedder import ClipEmbedder

embedder = ClipEmbedder()
device = 'cpu'

for fname in glob.glob('frames/*.jpg'):
    fname_w_o_extension = fname[:-4]
    image = embedder.preprocess(Image.open(f"frames/{fname}")).unsqueeze(0).to(device)
    image_features = embedder.encode_image(image)
    np.save(f"frame_vectors/{fname_w_o_extension}.npy", image_features.detach().numpy(), allow_pickle=False)